In [414]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pn


In [476]:
data = pn.read_csv('titanic.csv', index_col="PassengerId")

In [66]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 1
Какое количество мужчин и женщин ехало на корабле? В качестве
ответа приведите два числа через пробел.

In [55]:
male = data['Sex'].value_counts()['male']
female = data['Sex'].value_counts().female
print('female = ',female)
print('male = ',male)

female =  314
male =  577


### 2
Какой части пассажиров удалось выжить? 
Посчитайте долю выживших пассажиров. 
Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

In [101]:
ser = (data['Survived'].value_counts(normalize=True)[1])*100
ser.round(2)

38.38

### 3
Какую долю пассажиры первого класса составляли среди всех пассажиров? Ответ приведите в процентах (число в интервале от 0 до
100, знак процента не нужен), округлив до двух знаков.


In [87]:
d = data['Pclass'].value_counts(normalize=True)
d


3    0.551066
1    0.242424
2    0.206510
Name: Pclass, dtype: float64

In [88]:
((data['Pclass'] == 1).sum()/len(data)).round(2)

0.24

### 4
 Какого возраста были пассажиры? Посчитайте среднее и медиану возраста пассажиров. Посчитайте среднее и медиану возраста
пассажиров. В качестве ответа приведите два числа через пробел.

In [129]:
round(data['Age'].mean(),2)

29.7

In [130]:

round(data['Age'].median(),2)

28.0

### 5

Коррелируют ли число братьев/сестер/супругов с числом родителей/детей? Посчитайте корреляцию Пирсона между признаками SibSp и Parch.

In [164]:
data.corr(method='pearson')['SibSp']['Parch']

0.41483769862015624

### 6
Какое самое популярное женское имя на корабле? Извлеките из
полного имени пассажира (колонка Name) его личное имя (First
Name). Это задание — типичный пример того, с чем сталкивается специалист по анализу данных. Данные очень разнородные и
шумные, но из них требуется извлечь необходимую информацию.
Попробуйте вручную разобрать несколько значений столбца Name
и выработать правило для извлечения имен, а также разделения
их на женские и мужские

In [423]:
names = data.loc[(data['Sex'] == 'female')]['Name']
names

   

PassengerId
2      Cumings, Mrs. John Bradley (Florence Briggs Th...
3                                 Heikkinen, Miss. Laina
4           Futrelle, Mrs. Jacques Heath (Lily May Peel)
9      Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
10                   Nasser, Mrs. Nicholas (Adele Achem)
                             ...                        
881         Shelley, Mrs. William (Imanita Parrish Hall)
883                         Dahlberg, Miss. Gerda Ulrika
886                 Rice, Mrs. William (Margaret Norton)
888                         Graham, Miss. Margaret Edith
889             Johnston, Miss. Catherine Helen "Carrie"
Name: Name, Length: 314, dtype: object

In [330]:
name = list()
for i in names:
    if 'Miss.' in i:
        ind = i.index("Miss.")
        name.append( i[ind+1])
    else:
        for c in i:
            if c[0] == '(':
                name.append(c)
                
new = list()
for i in name:
    if '(' or ')' in i:
        i = i.replace('(', '')
        i = i.replace(')', '')
    new.append(i)


308

In [ ]:
'Miss. Name'

In [349]:
def get_name(full_name):
    miss_ind = full_name.find('Miss.')
    name = ''
    
    if miss_ind != -1:
        name = full_name[miss_ind + 6:].split(' ')[0]
    else:
        bracket_ind = full_name.find('(')
        
        if bracket_ind != -1:
            name = full_name[bracket_ind+1:].split(' ')[0].replace(')', '')
    
    return name

In [375]:
women = data[data['Sex'] == 'female'][['Name']]
women['true_name'] = women['Name'].apply(get_name)
women['true_name'].value_counts(ascending=False)

Anna         15
Mary         14
Elizabeth    11
Margaret     10
              7
             ..
Manca         1
Miriam        1
Hildur        1
Delia         1
Emma          1
Name: true_name, Length: 175, dtype: int64

In [333]:
new_f = pn.DataFrame(new)
new_f.value_counts()

Anna         15
Mary         14
Elizabeth    11
Margaret     10
Alice         6
             ..
Eva           1
Eugenie       1
Esther        1
Erna          1
Wendla        1
Length: 173, dtype: int64

# решающие деревья



In [485]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [486]:
new_frame = data[['Sex','Age','Pclass','Fare','Survived']]
new_frame = new_frame.dropna()
new_frame

,Sex,Age,Pclass,Fare,Survived
PassengerId,,,,,
1,male,22.0,3,7.2500,0
2,female,38.0,1,71.2833,1
3,female,26.0,3,7.9250,1
4,female,35.0,1,53.1000,1
5,male,35.0,3,8.0500,0
...,...,...,...,...,...
886,female,39.0,3,29.1250,0
887,male,27.0,2,13.0000,0
888,female,19.0,1,30.0000,1


In [487]:
def fun(str):
    if str == 'male':
        return 0
    else:
        return 1
new_frame['Sex'] = new_frame['Sex'].apply(fun)
new_frame

,Sex,Age,Pclass,Fare,Survived
PassengerId,,,,,
1,0,22.0,3,7.2500,0
2,1,38.0,1,71.2833,1
3,1,26.0,3,7.9250,1
4,1,35.0,1,53.1000,1
5,0,35.0,3,8.0500,0
...,...,...,...,...,...
886,1,39.0,3,29.1250,0
887,0,27.0,2,13.0000,0
888,1,19.0,1,30.0000,1


In [507]:
y = np.array(new_frame['Survived'])
y
y

array([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,

In [489]:
sex = np.array(new_frame['Sex'])
age = np.array(new_frame['Age'])
Pclass = np.array(new_frame['Pclass'])
Fare = np.array(new_frame['Fare'])


In [503]:
from sklearn.tree import DecisionTreeClassifier
X = np.array([sex, age, Pclass,Fare])
X
X = X.transpose()
np.shape(X)
X

array([[ 0.    , 22.    ,  3.    ,  7.25  ],
       [ 1.    , 38.    ,  1.    , 71.2833],
       [ 1.    , 26.    ,  3.    ,  7.925 ],
       ...,
       [ 1.    , 19.    ,  1.    , 30.    ],
       [ 0.    , 26.    ,  1.    , 30.    ],
       [ 0.    , 32.    ,  3.    ,  7.75  ]])

In [508]:
clf = DecisionTreeClassifier(random_state=241)
clf.fit(X, y)

DecisionTreeClassifier(random_state=241)

In [509]:
importances = clf.feature_importances_
importances

array([0.30051221, 0.25658495, 0.14751816, 0.29538468])